In [28]:
!conda install -y -n canarydev redis 

In [20]:
!npm install -g kue-ui kue

In [109]:
%%file lognohup
#!/usr/bin/env bash
nohup "$@" 1>$LOGFNAME.log 2>$LOGFNAME.err &

#!/usr/bin/env bash
nohup "$@" 1>$LOGFNAME.log 2>$LOGFNAME.err &



In [ ]:
!chmod +x lognohup

In [34]:
!LOGFNAME=redis-server lognohup redis-server
!LOGFNAME=kui-ui lognohup node kui-ui.js

In [137]:
!tail -n 80 kui-ui.??? redis-server.???

In [46]:
!LOGFNAME=kuilist lognohup node listJobs.js

In [58]:
!kill 157528

In [98]:
!pgrep -af listJobs
!tail kuilist*

==> kuilist.err <==

==> kuilist.log <==
Job 112, data : [object Object], Status : complete
Job 113, data : [object Object], Status : complete
Job 114, data : [object Object], Status : complete
Job 115, data : [object Object], Status : complete
Job 116, data : [object Object], Status : complete
Job 117, data : [object Object], Status : complete
Job 118, data : [object Object], Status : complete
Job 119, data : [object Object], Status : complete
Job 120, data : [object Object], Status : complete
Job 121, data : [object Object], Status : complete


In [63]:
!LOGFNAME=kuiworker0 lognohup node worker.js
!LOGFNAME=kuiworker1 lognohup node worker.js
!LOGFNAME=kuiworker2 lognohup node worker.js
!LOGFNAME=kuiworker3 lognohup node worker.js

In [12]:
!pgrep -af worker.js
!tail -n 20 kuiworker?.???

157708 node worker.js
157715 node worker.js
157723 node worker.js
157731 node worker.js
==> kuiworker0.err <==

==> kuiworker0.log <==
Download Complete
Working on job 185
{ file:
   'Sample/path/to/fileWed Dec 11 2019 19:38:01 GMT+0800 (China Standard Time)' }
2019-12-11T11:38:02.005Z
Download Complete
Working on job 189
{ file:
   'Sample/path/to/fileWed Dec 11 2019 19:39:04 GMT+0800 (China Standard Time)' }
2019-12-11T11:39:04.770Z
Download Complete
Working on job 192
{ file:
   'Sample/path/to/fileWed Dec 11 2019 19:39:45 GMT+0800 (China Standard Time)' }
2019-12-11T11:39:45.267Z
Download Complete

==> kuiworker1.err <==

==> kuiworker1.log <==
Download Complete
Working on job 184
{ file:
   'Sample/path/to/fileWed Dec 11 2019 19:38:01 GMT+0800 (China Standard Time)' }
2019-12-11T11:38:02.005Z
Download Complete
Working on job 188
{ file:
   'Sample/path/to/fileWed Dec 11 2019 19:39:04 GMT+0800 (China Standard Time)' }
2019-12-11T11:39:04.770Z
Download Complete
Working on job 193
{ 

In [3]:
!node client.js

2019-12-11T11:39:45.256Z
2019-12-11T11:39:45.259Z
2019-12-11T11:39:45.259Z
2019-12-11T11:39:45.259Z
Job Submitted in the Queue.


In [61]:
!kill `pgrep -f worker.js`

In [87]:
!ps -aux |egrep 'node|CPU' |grep -v coffee

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
mqq         659  0.0  0.0 621088 14708 ?        Sl   Dec06   0:00 ./node ./tsw
mqq        1115  0.0  0.0 899540 33728 ?        Sl   Dec06   0:22 /usr/local/app/canary/bin/node /usr/local/app/canary/lib/node_modules/chp/bin/chp
mqq        2494  0.0  0.0 563248 16660 ?        Sl   Dec06   0:00 node gitsvr.js
mqq      156226  0.0  0.0 606652 47324 ?        Sl   18:12   0:01 node kui-ui.js
mqq      157708  0.2  0.0 610580 43508 ?        Sl   19:32   0:00 node worker.js
mqq      157715  0.2  0.0 610560 43280 ?        Sl   19:32   0:00 node worker.js
mqq      157723  0.2  0.0 610600 43252 ?        Sl   19:32   0:00 node worker.js
mqq      157731  0.2  0.0 610560 43548 ?        Sl   19:32   0:00 node worker.js
mqq      157882  0.0  0.0 110396   900 pts/22   S+   19:36   0:00 grep -E node|CPU


In [13]:
!git diff

diff --git a/client.js b/client.js
index 6e9bdf6..6809a83 100644
--- a/client.js
+++ b/client.js
@@ -1,20 +1,21 @@
 let kue   = require(`kue`);
-
 let queue = kue.createQueue();
 
+var i;
 //event handler. Called when job is saved to the Redis.
 queue.on(`job enqueue`, function(){
     console.log(`Job Submitted in the Queue.`);
-    process.exit(0);
+    if (i==4)
+        process.exit(0);
 });
 
-let job = queue.create(`download`, {
-  file : `Sample/path/to/file`
-})
-.attempts(3) // if job fails retry 3 times
-.backoff({delay: 60 * 1000}) // wait 60s before retry
-.save();
-
-
+for (i = 0; i < 4; i++) {
+    let job = queue.create(`download`, {
+        file : `Sample/path/to/file${new Date()}`
+    })
+    .attempts(3) // if job fails retry 3 times
+    .backoff({delay: 60 * 1000}) // wait 60s before retry
+    .save();
 
- 
+    console.log(new Date());
+}
diff --git a/listJobs.js b/listJobs.js
index b1c6a32..410322d 100644
--- a/listJobs.js
+++ b/listJobs.js
@@ -7,10 +7,10 @@ ku

In [ ]:
!git add client.js worker.js kuetest.ipynb